## Hands-on Modul 3: Pipeline Inference Dasar (PyTorch & Hugging Face)
_Repository GitHub: https://github.com/Komdigi-x-BINAR-LLM/Module-03_

**Tujuan:** Mempraktikkan alur kerja dasar dari teks mentah hingga mendapatkan output internal model (`last_hidden_state`).

In [2]:
# Setup Awal (Jalankan sel ini jika di Colab atau environment baru)
# !pip install transformers torch notebook

In [3]:
# Import library
from transformers import AutoTokenizer, AutoModel # Gunakan AutoModel dasar
import torch
import logging
import sys
import importlib

# --- Force re-configuration logging for notebooks ---
importlib.reload(logging)
logging.basicConfig(
    level=logging.INFO, # Tampilkan pesan INFO
    format='%(levelname)s: %(message)s',
    stream=sys.stdout # Paksa output ke stdout (yang ditangkap notebook)
)
# -----------------------------

logging.info("Setup selesai. Library siap digunakan.")

INFO: Setup selesai. Library siap digunakan.


---
### Langkah 1: Load Model & Tokenizer (Konsep 3.2)
Kita akan memuat model Encoder-only yang ringan (DistilBERT) dan tokenizer-nya.

In [4]:
model_name = "distilbert-base-uncased" # Model Encoder-only ringan
logging.info(f"Mencoba memuat tokenizer untuk '{model_name}'...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    logging.info("Tokenizer berhasil dimuat.")
except Exception as e:
    logging.error(f"Gagal memuat tokenizer: {e}")
    raise

logging.info(f"Mencoba memuat model '{model_name}'...")
try:
    model = AutoModel.from_pretrained(model_name)
    logging.info("Model berhasil dimuat.")
except Exception as e:
    logging.error(f"Gagal memuat model: {e}")
    raise

INFO: Mencoba memuat tokenizer untuk 'distilbert-base-uncased'...
INFO: Tokenizer berhasil dimuat.
INFO: Mencoba memuat model 'distilbert-base-uncased'...
INFO: loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/model.safetensors
INFO: Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassificat

---
### Langkah 2: Siapkan Device (Konsep 3.1)
Kita cek ketersediaan GPU dan pindahkan model ke device yang sesuai.

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    logging.info("GPU (cuda) terdeteksi. Model akan dipindahkan ke GPU.")
else:
    device = torch.device("cpu")
    logging.info("GPU tidak terdeteksi. Model akan menggunakan CPU.")

# Pindahkan model
model.to(device)
logging.info(f"Model sekarang berada di device: {model.device}")

INFO: GPU tidak terdeteksi. Model akan menggunakan CPU.
INFO: Model sekarang berada di device: cpu


---
### Langkah 3: Siapkan Input & Preprocessing (Konsep 3.3)
Kita akan memproses beberapa kalimat dengan panjang berbeda menggunakan tokenizer,
menerapkan padding dan truncation, serta menghasilkan PyTorch tensor.

In [6]:
kalimat_batch = [
    "Hugging Face itu keren!", # Kalimat pendek
    "Ini adalah kalimat kedua yang sedikit lebih panjang.", # Kalimat lebih panjang
    "Contoh kalimat ketiga ini dibuat sangat panjang sekali agar melebihi batas umum beberapa model dasar dan kita bisa melihat efek truncation jika diaktifkan nanti." # Kalimat sangat panjang
]
print(f"Input Teks (Batch):\n{kalimat_batch}")

# Tentukan max_length (misal, 20 untuk demonstrasi truncation)
max_len = 20
logging.info(f"Melakukan tokenisasi dengan padding=True, truncation=True, max_length={max_len}, return_tensors='pt'...")

inputs = tokenizer(
    kalimat_batch,
    padding=True,       # Aktifkan dynamic padding
    truncation=True,    # Aktifkan truncation
    max_length=max_len, # Batasi panjang maksimal
    return_tensors="pt" # Minta output sebagai PyTorch Tensor
)

logging.info("Tokenisasi selesai.")
print("\nHasil Tokenisasi (Dictionary Tensor):")
print("{")
for key, tensor in inputs.items():
    print(f"  '{key}':")
    print(f"    Shape: {tensor.shape}")
    print(f"    Tensor:\n{tensor}")
print("}")

Input Teks (Batch):
['Hugging Face itu keren!', 'Ini adalah kalimat kedua yang sedikit lebih panjang.', 'Contoh kalimat ketiga ini dibuat sangat panjang sekali agar melebihi batas umum beberapa model dasar dan kita bisa melihat efek truncation jika diaktifkan nanti.']
INFO: Melakukan tokenisasi dengan padding=True, truncation=True, max_length=20, return_tensors='pt'...
INFO: Tokenisasi selesai.

Hasil Tokenisasi (Dictionary Tensor):
{
  'input_ids':
    Shape: torch.Size([3, 20])
    Tensor:
tensor([[  101, 17662,  2227,  2009,  2226, 17710,  7389,   999,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1999,  2072, 15262, 14431, 19924, 18900, 16135,  6692,  8675,
          7367,  4305, 23615,  3393,  5638,  2232,  6090,  8405,  2290,   102],
        [  101,  9530,  3406,  2232, 19924, 18900, 17710,  3775,  3654,  1999,
          2072,  4487,  8569,  4017,  6369,  4017,  6090,  8405,  2290,   102]])
  'attention_mask':
    S

**Observasi Hasil Tokenisasi:**
* Perhatikan `shape` dari `input_ids` dan `attention_mask`. Angka terakhir (kolom) harusnya sama untuk semua kalimat dan tidak lebih dari `max_length`.
* Lihat nilai `input_ids`. Apakah ada angka `0` (ID token [PAD]) di kalimat yang lebih pendek?
* Lihat nilai `attention_mask`. Apakah ada angka `0` di posisi yang sama dengan token [PAD]?
* Apakah kalimat yang sangat panjang terpotong?

---
### Langkah 4: Jalankan Inference (Konsep 3.4)
Kita pindahkan input tensor ke device yang sama dengan model dan jalankan inference.

In [7]:
# Pindahkan input tensor ke device
logging.info(f"Memindahkan input tensors ke {device}...")
try:
    inputs_on_device = {k: v.to(device) for k, v in inputs.items()}
    logging.info("Input berhasil dipindahkan.")

    # Jalankan inference dalam context torch.no_grad()
    logging.info("Menjalankan inference model...")
    with torch.no_grad():
        outputs = model(**inputs_on_device) # Unpack dictionary inputs
    logging.info("Inference selesai.")

except Exception as e:
    logging.error(f"Terjadi error saat memindahkan input atau inference: {e}", exc_info=True)
    outputs = None # Set outputs ke None jika gagal

INFO: Memindahkan input tensors ke cpu...
INFO: Input berhasil dipindahkan.
INFO: Menjalankan inference model...
INFO: Inference selesai.


---
### Langkah 5: Analisis Output (Konsep 3.1 & 3.4)
Kita periksa output utama dari `AutoModel` dasar, yaitu `last_hidden_state`.

In [8]:
print("\n### Analisis Output Model ###")
if outputs is not None and hasattr(outputs, 'last_hidden_state'):
    last_hidden_states = outputs.last_hidden_state
    logging.info("Mengakses 'last_hidden_state' dari output.")

    print(f"\nShape dari last_hidden_state: {last_hidden_states.shape}")
    print(f"Tipe data (dtype): {last_hidden_states.dtype}")
    print(f"Device: {last_hidden_states.device}")

    # Penjelasan Shape: [Batch Size, Sequence Length, Hidden Size]
    # - Batch Size: Jumlah kalimat dalam input (harusnya 3).
    # - Sequence Length: Panjang token setelah padding/truncation (harusnya sama dengan max_len=20).
    # - Hidden Size: Dimensi vektor embedding internal model (768 untuk distilbert-base-uncased).
    print("\nPenjelasan Shape:")
    print(f"- Dimensi 0 (Batch Size): {last_hidden_states.shape[0]} (Jumlah kalimat)")
    print(f"- Dimensi 1 (Sequence Length): {last_hidden_states.shape[1]} (Panjang token)")
    print(f"- Dimensi 2 (Hidden Size): {last_hidden_states.shape[2]} (Dimensi embedding model)")

else:
    print("\nTidak dapat menganalisis output karena inference gagal atau output tidak sesuai.")


### Analisis Output Model ###
INFO: Mengakses 'last_hidden_state' dari output.

Shape dari last_hidden_state: torch.Size([3, 20, 768])
Tipe data (dtype): torch.float32
Device: cpu

Penjelasan Shape:
- Dimensi 0 (Batch Size): 3 (Jumlah kalimat)
- Dimensi 1 (Sequence Length): 20 (Panjang token)
- Dimensi 2 (Hidden Size): 768 (Dimensi embedding model)


**Eksperimen:**
* Coba ganti `model_name` ke model lain (misal: "bert-base-uncased") di Langkah 1. Apakah `Hidden Size` berubah?
* Ubah nilai `max_len` di Langkah 3. Apakah `Sequence Length` di output berubah?
* Coba masukkan kalimat Anda sendiri di `kalimat_batch`.